In [7]:
import torch
import torch.nn as nn
from rotary_embedding_torch import RotaryEmbedding

In [193]:

class AstromerEncoder(nn.Module):
    def __init__(self, time, embed_mag, ,mask, num_heads=3,embed_type='sincos', arg=None, dim=None, device="cuda"):
        super(AstromerEncoder, self).__init__()
        self.embed_mag = embed_mag
        self.device = device
        self.arg = arg
        self.embed_type = embed_type
        self.num_heads = num_heads
        self.mask = mask
        # -------------
        # need to project by attention heads first thing
        if embed_type == 'sincos':
            i = torch.arange(embed_mag)[:, None]
            self.w =  (1 / 1000 ** ((2 * i) / embed_mag)).repeat(1,100)
            self.w[::2] = torch.sin(self.w[::2]*time)
            self.w[1::2] = torch.cos(self.w[1::2]*time)
        # -------------
        else:
            self.rotary_emb = RotaryEmbedding(dim=embed_mag)
        self.linears = nn.Sequential(
            nn.Linear(1, embed_mag, bias=False),
            nn.Linear(embed_mag, embed_mag, bias=False),
            nn.Linear(embed_mag, embed_mag, bias=False)
        ) 
        
        
        ## sin,cos addition is made relative to mag values
    def attention(self, q,k,v,mask=None):
        scores = torch.matmul(q,k.T) / self.embed_mag
        # scores = scores.masked_fill(mask.unsqueeze(-3) == 0, -1e9) # before softmax!
        
        weights = self.softmax(scores)
        out = torch.matmul(weights, v)
        
        return out
    
    
        #scores = torch.softmax
    def forward(self, tt, mags):
        ## pass the magnitudes through a few layers to project them
        ## add positional encoding/ rotary encoding 
        mags = self.linears(mags)
        if self.embed_type == 'sincos':
            mags += self.w.T
            
        else:
            mags = self.rotary_emb.rotate_queries_or_keys(mags) # meant to be input with seq_length, feature dim as the last 2 dims
        
        out = self.attention(mags,mags,mags)
        return out



In [194]:
import torch
time = torch.arange(0,1,0.01)[None, :]
print(time.shape)
mags = torch.rand(100)[:,None]
print(x)
l = AstromerEmbedding(time, 8)# embed_type='rotary')


torch.Size([1, 100])
tensor([[0.0108, 0.0105, 0.0082,  ..., 0.0095, 0.0091, 0.0127],
        [0.0108, 0.0105, 0.0082,  ..., 0.0095, 0.0092, 0.0127],
        [0.0107, 0.0105, 0.0089,  ..., 0.0096, 0.0094, 0.0116],
        ...,
        [0.0074, 0.0073, 0.0058,  ..., 0.0123, 0.0120, 0.0165],
        [0.0074, 0.0073, 0.0058,  ..., 0.0124, 0.0120, 0.0163],
        [0.0074, 0.0072, 0.0052,  ..., 0.0122, 0.0117, 0.0182]],
       grad_fn=<SoftmaxBackward0>)


In [196]:
out = l.forward(time, mags)

torch.Size([100, 100])


In [197]:
x = nn.Softmax(dim=1)(out)
x[0]

tensor([0.0104, 0.0107, 0.0083, 0.0085, 0.0098, 0.0094, 0.0089, 0.0082, 0.0090,
        0.0111, 0.0087, 0.0100, 0.0081, 0.0110, 0.0087, 0.0096, 0.0083, 0.0091,
        0.0085, 0.0104, 0.0113, 0.0098, 0.0107, 0.0083, 0.0091, 0.0092, 0.0099,
        0.0105, 0.0092, 0.0104, 0.0089, 0.0086, 0.0113, 0.0099, 0.0101, 0.0100,
        0.0095, 0.0115, 0.0111, 0.0090, 0.0117, 0.0106, 0.0106, 0.0116, 0.0110,
        0.0084, 0.0113, 0.0104, 0.0089, 0.0089, 0.0117, 0.0097, 0.0092, 0.0108,
        0.0087, 0.0105, 0.0095, 0.0097, 0.0088, 0.0105, 0.0108, 0.0094, 0.0102,
        0.0089, 0.0090, 0.0107, 0.0115, 0.0116, 0.0086, 0.0095, 0.0107, 0.0116,
        0.0095, 0.0104, 0.0116, 0.0099, 0.0101, 0.0086, 0.0099, 0.0101, 0.0117,
        0.0117, 0.0098, 0.0105, 0.0085, 0.0110, 0.0100, 0.0117, 0.0085, 0.0120,
        0.0117, 0.0110, 0.0089, 0.0116, 0.0098, 0.0105, 0.0092, 0.0097, 0.0114,
        0.0109], grad_fn=<SelectBackward0>)

In [ ]:

#### connvvvvv layyyyyyerrrrzzzzzz
class ACEmbedding(nn.Module):
    def __init__(self, embed_mag, arg=None, dim=None, device="cuda"):
        super(ACEmbedding, self).__init__()
        self.embed_mag = embed_mag
    def forward(self, tt, mags):
        